<a href="https://colab.research.google.com/github/dbtmddn41/deep_learning/blob/main/practice/text_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-05-25 18:05:07--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  20.4MB/s    in 6.9s    

2023-05-25 18:05:14 (11.7 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
dataset = keras.utils.text_dataset_from_directory(directory='aclImdb', label_mode=None, batch_size = 64)
dataset = dataset.map(lambda x: tf.strings.regex_replace(x, "<br />", " "))

Found 100006 files belonging to 1 classes.


In [ ]:
from keras.layers import TextVectorization

sequence_len = 100
vocab_size = 15000
text_vectorization = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=sequence_len)
text_vectorization.adapt(dataset)

In [ ]:
def prepare_lm_dataset(text_batch):
    vectorized_sequences = text_vectorization(text_batch)
    x = vectorized_sequences[:, :-1]
    y = vectorized_sequences[:, 1:]
    return x, y

lm_dataset = dataset.map(prepare_lm_dataset, num_parallel_calls=16)

In [ ]:
class TransformerEncoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=self.dropout_rate)
        self.dense_proj = keras.Sequential([layers.Dense(dense_dim, activation='relu'), layers.Dense(embed_dim)])
        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(query=inputs, value=inputs, attention_mask=mask)
        x = inputs + self.dropout1(attention_output)
        x = self.layernorm_1(x)
        proj_output = self.dense_proj(x)
        x = self.dropout2(proj_output) + x
        output = self.layernorm_2(x)

        return output

    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads, 'dropout_rate': self.dropout_rate})
        return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_len, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(input_dim=input_dim, output_dim=output_dim)#, embeddings_initializer=keras.initializers.Constant(embedding_matrix), trainable=False)
        self.position_embeddings = layers.Embedding(input_dim=sequence_len, output_dim=output_dim)
        self.sequence_len = sequence_len
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({"sequence_len": self.sequence_len, "input_dim": self.input_dim, "output_dim": self.output_dim})
        return config

class TransformerEncoder(layers.Layer):
    def __init__(self, num_blocks, sequence_len, vocab_size, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.num_blocks = num_blocks
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.pos_embedding = PositionalEmbedding(sequence_len, vocab_size, embed_dim)
        block_layers = []
        for i in range(num_blocks):
            block_layers.append(TransformerEncoderBlock(embed_dim, dense_dim, num_heads, dropout_rate))
        self.block_layers = keras.Sequential(block_layers)

    def call(self, inputs):
        pos_inputs = self.pos_embedding(inputs)
        encoded = self.block_layers(pos_inputs)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_blocks': self.num_blocks, 'sequence_len': self.sequence_len,
            'vocab_size': self.vocab_size, 'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim, 'num_heads': self.num_heads,
            'dropout_rate': self.dropout_rate
        })
        return config

class TransformerDecoderBlock(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.attention_1 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.attention_2 = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate)
        self.dense_proj = keras.Sequential([
            layers.Dense(dense_dim, activation='relu'),
            layers.Dense(embed_dim)
        ])

        self.dropout1 = layers.Dropout(dropout_rate)
        self.dropout2 = layers.Dropout(dropout_rate)
        self.dropout3 = layers.Dropout(dropout_rate)

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({"embed_dim": self.embed_dim, "dense_dim": self.dense_dim, "num_heads": self.num_heads, 'dropout_rate': self.dropout_rate})
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_len = input_shape[0], input_shape[1]
        i = tf.range(sequence_len)[:, tf.newaxis]
        j = tf.range(sequence_len)
        mask = tf.cast(i >= j, dtype='int32')
        mask = tf.reshape(mask, (1, sequence_len, sequence_len))
        mult = tf.concat([tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype='int32')
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(query=inputs, key=inputs, value=inputs, attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(self.dropout1(attention_output_1)+inputs)
        attention_output_2 = self.attention_2(query=attention_output_1, key=encoder_outputs, value=encoder_outputs, attention_mask=padding_mask)
        attention_output_2 = self.layernorm_2(self.dropout2(attention_output_2) + attention_output_1)
        proj_output = self.dense_proj(attention_output_2)
        outputs = self.layernorm_3(attention_output_2 + self.dropout3(proj_output))
        return outputs

class TransformerDecoder(layers.Layer):
    def __init__(self, num_blocks, sequence_len, vocab_size, embed_dim, dense_dim, num_heads, dropout_rate=0.0, **kwargs):
        super().__init__(**kwargs)
        self.num_blocks = num_blocks
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.dropout_rate = dropout_rate

        self.pos_embedding = PositionalEmbedding(sequence_len, vocab_size, embed_dim)
        self.block_layers = []
        for i in range(num_blocks):
            self.block_layers.append(TransformerDecoderBlock(embed_dim, dense_dim, num_heads, dropout_rate))
        self.dropout = layers.Dropout(dropout_rate)
        self.dense = layers.Dense(vocab_size, activation='softmax')

    def call(self, inputs, encoder_outputs, self_encode=False):
        x = self.pos_embedding(inputs)
        if self_encode:
            encoder_outputs = x
        for decoder_block in self.block_layers:
            x = decoder_block(x, encoder_outputs)
        x = self.dropout(x)
        outputs = self.dense(x)
        return outputs

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_blocks': self.num_blocks, 'sequence_len': self.sequence_len,
            'vocab_size': self.vocab_size, 'embed_dim': self.embed_dim,
            'dense_dim': self.dense_dim, 'num_heads': self.num_heads,
            'dropout_rate': self.dropout_rate
            })
        return config

In [ ]:
num_blocks = 5
embed_dim = 256
dense_dim = 2048
num_heads = 34


decoder_inputs = keras.Input(shape=(sequence_len-1), dtype='int32', name='english')
decoder_outputs = TransformerDecoder(num_blocks, sequence_len, vocab_size, embed_dim, dense_dim, num_heads, 0.1)(decoder_inputs, None, self_encode=True)
transformer_decoder = keras.Model(decoder_inputs, decoder_outputs, name='transformer_decoder')
transformer_decoder.summary()
transformer_decoder.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

Model: "transformer_decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 english (InputLayer)        [(None, 99)]              0         
                                                                 
 transformer_decoder (Transf  (None, 99, 15000)        102375320 
 ormerDecoder)                                                   
                                                                 
Total params: 102,375,320
Trainable params: 102,375,320
Non-trainable params: 0
_________________________________________________________________


In [ ]:
text_vectorization(['hello']).shape

TensorShape([1, 100])

In [ ]:
import numpy as np
from tqdm import tqdm

vocab = text_vectorization.get_vocabulary()
tokens_index_lookup = np.pad(np.array(vocab), pad_width=(0,vocab_size), mode='constant', constant_values='[UNK]')[:vocab_size]

def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature
    exp_preds = np.exp(predictions)
    predictions = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, predictions, 1)
    return np.argmax(probas)

class TextGenerator(keras.callbacks.Callback):
    def __init__(self, prompt, generate_length, model_input_length, temperatures=(1.,), print_freq=1):
        self.prompt = prompt
        self.generate_length = generate_length
        self.model_input_length = model_input_length
        self.temperatures = temperatures
        self.print_freq = print_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % self.print_freq != 0:
            return
        for temperature in self.temperatures:
            sentence = self.prompt
            tokenized_sentence = text_vectorization([sentence])[:, :-1]
            tokenized_sentence = tf.Variable(initial_value=tokenized_sentence)
            for i in tqdm(range(self.generate_length), desc=f"===={temperature}의 온도로 생성 중"):
                predictions = self.model.predict(tokenized_sentence, verbose=False)
                next_token = sample_next(predictions[0, i, :])
                sampled_token = tokens_index_lookup[next_token]
                sentence += " " + sampled_token
                if i < sequence_len-1:
                    tokenized_sentence[0, i+1, tf.newaxis].assign(next_token)
            print(sentence)

In [ ]:
prompt = "This movie"
text_gen_callback = TextGenerator(prompt, generate_length=50, model_input_length=sequence_len, temperatures=(0.2, 0,5, 0.7, 1., 1.5), print_freq=25)

In [ ]:
for x, y in lm_dataset.take(1):
    print(tf.shape(x), tf.shape(y))

tf.Tensor([256  99], shape=(2,), dtype=int32) tf.Tensor([256  99], shape=(2,), dtype=int32)


In [ ]:
transformer_decoder.fit(lm_dataset.prefetch(8), epochs=200, callbacks=[text_gen_callback])

Epoch 1/200
1563/1563 [==============================] - 860s 541ms/step - loss: 5.6346
Epoch 2/200
1563/1563 [==============================] - 845s 540ms/step - loss: 5.0039
Epoch 3/200
 381/1563 [======>.......................] - ETA: 10:38 - loss: 4.8838